In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
display_table('Delivery')

+-------------+-------------+------------+-----------------------------+
| delivery_id | customer_id | order_date | customer_pref_delivery_date |
+-------------+-------------+------------+-----------------------------+
|      1      |      1      | 2019-08-01 |          2019-08-02         |
|      2      |      2      | 2019-08-02 |          2019-08-02         |
|      3      |      1      | 2019-08-11 |          2019-08-12         |
|      4      |      3      | 2019-08-24 |          2019-08-24         |
|      5      |      3      | 2019-08-21 |          2019-08-22         |
|      6      |      2      | 2019-08-11 |          2019-08-13         |
|      7      |      4      | 2019-08-09 |          2019-08-09         |
+-------------+-------------+------------+-----------------------------+



### Write an SQL query to find the percentage of immediate orders in the first orders of all customers, rounded to 2 decimal places.

#### *If the customer's preferred delivery date is the same as the order date, then the order is called immediate; otherwise, it is called scheduled.* *The first order of a customer is the order with the earliest order date that the customer made. It is guaranteed that a customer has precisely one first order.*
```
+----------------------+
| immediate_percentage |
+----------------------+
| 50.00                |
+----------------------+
Explanation: 
The customer id 1 has a first order with delivery id 1 and it is scheduled.
The customer id 2 has a first order with delivery id 2 and it is immediate.
The customer id 3 has a first order with delivery id 5 and it is scheduled.
The customer id 4 has a first order with delivery id 7 and it is immediate.
Hence, half the customers have immediate first orders.
```


In [21]:
%%sql 

SELECT 
    COUNT(*)
FROM Delivery 
WHERE order_date = customer_pref_delivery_date

COUNT(*)
3


In [3]:
%%sql 

SELECT customer_id, MIN(order_date) AS earliest_order
FROM Delivery
GROUP BY customer_id

customer_id,earliest_order
1,2019-08-01
2,2019-08-02
3,2019-08-21
4,2019-08-09


In [4]:
%%sql 
-- the PARTITION BY clause specifies the column (customer_id) 
-- by which the rows should be grouped, 
-- AND the MIN() function calculates the minimum value within each group.
SELECT
    customer_id,
    order_date,
    customer_pref_delivery_date,
    MIN(order_date) OVER (PARTITION BY customer_id) AS earliest_order
FROM Delivery


customer_id,order_date,customer_pref_delivery_date,earliest_order
1,2019-08-01,2019-08-02,2019-08-01
1,2019-08-11,2019-08-12,2019-08-01
2,2019-08-02,2019-08-02,2019-08-02
2,2019-08-11,2019-08-13,2019-08-02
3,2019-08-24,2019-08-24,2019-08-21
3,2019-08-21,2019-08-22,2019-08-21
4,2019-08-09,2019-08-09,2019-08-09


In [5]:
%%sql 

SELECT 
    delivery_id,
    MIN(order_date) OVER (PARTITION BY customer_id) AS earliest_order
FROM Delivery
GROUP BY delivery_id

delivery_id,earliest_order
1,2019-08-01
3,2019-08-01
2,2019-08-02
6,2019-08-02
4,2019-08-21
5,2019-08-21
7,2019-08-09


In [6]:
%%sql 

WITH cte_table AS (
    SELECT 
        delivery_id,
        MIN(order_date) OVER (PARTITION BY customer_id) AS earliest_order
    FROM Delivery
    GROUP BY delivery_id
)
SELECT 
    Delivery.delivery_id,
    Delivery.order_date,
    Delivery.customer_pref_delivery_date,
    cte_table.earliest_order
FROM Delivery
JOIN cte_table ON cte_table.delivery_id = Delivery.delivery_id
WHERE customer_pref_delivery_date = earliest_order

delivery_id,order_date,customer_pref_delivery_date,earliest_order
2,2019-08-02,2019-08-02,2019-08-02
7,2019-08-09,2019-08-09,2019-08-09


In [23]:
%%sql 

WITH cte_table AS (
    SELECT 
        delivery_id,
        MIN(order_date) OVER (PARTITION BY customer_id) AS earliest_order
    FROM Delivery
    GROUP BY delivery_id
)
SELECT 
    COUNT(Delivery.delivery_id) AS immediate_order_count
FROM Delivery
JOIN cte_table ON cte_table.delivery_id = Delivery.delivery_id
WHERE customer_pref_delivery_date = earliest_order

immediate_order_count
2


In [24]:
%%sql 

WITH cte_table AS (
    SELECT 
        delivery_id,
        MIN(order_date) OVER (PARTITION BY customer_id) AS earliest_order
    FROM Delivery
    GROUP BY delivery_id
)
SELECT 
    COUNT(Delivery.delivery_id) / (SELECT COUNT(DISTINCT customer_id) FROM Delivery) AS immediate_percentage
FROM Delivery
JOIN cte_table ON cte_table.delivery_id = Delivery.delivery_id
WHERE customer_pref_delivery_date = earliest_order

immediate_percentage
0.5000


In [25]:
%%sql 

WITH cte_table AS (
    SELECT 
        delivery_id,
        MIN(order_date) OVER (PARTITION BY customer_id) AS earliest_order
    FROM Delivery
    GROUP BY delivery_id
)
SELECT 
    ROUND((COUNT(Delivery.delivery_id) / (SELECT COUNT(DISTINCT customer_id) FROM Delivery)) * 100, 2) AS immediate_percentage
FROM Delivery
JOIN cte_table ON cte_table.delivery_id = Delivery.delivery_id
WHERE customer_pref_delivery_date = earliest_order

immediate_percentage
50.00


### Solution without using CTE

In [26]:
%%sql 

SELECT
    ROUND((COUNT(*) / (SELECT COUNT(DISTINCT customer_id) FROM Delivery)) * 100, 2) AS immediate_percentage
FROM Delivery
WHERE order_date = (
    SELECT MIN(order_date)
    FROM Delivery d
    WHERE d.customer_id = Delivery.customer_id
)
AND order_date = customer_pref_delivery_date;


immediate_percentage
50.00
